In [1]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Set constants for environment
import os
import sys

IN_COLAB = 'google.colab' in sys.modules
IN_LOCAL_CONTAINER = 'HOMELAB' in os.environ
if IN_COLAB:
    root_dir = os.path.join("/content", "lvgl-ui-detector")
elif IN_LOCAL_CONTAINER:
    root_dir = os.path.join(os.environ['HOME'], 'project')
else:
    root_dir = os.path.join(os.environ['HOME'], 'lvgl-ui-detector')
data_dir = os.path.join(root_dir, 'tmp', 'custom')
if not IN_COLAB:
    randomizer_dir = os.path.join(root_dir, 'ui_randomizer')
    generator_binary_dir = os.path.join(root_dir, 'lvgl_ui_generator', 'build', 'bin')
    generator_binary = os.path.join(generator_binary_dir, 'demo')
custom_dataset_file = os.path.join(data_dir, 'custom.yaml')

In [1]:
! pip show ultralytics

Name: ultralytics
Version: 8.1.1
Summary: Ultralytics YOLOv8 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: 
Author: Glenn Jocher, Ayush Chaurasia, Jing Qiu
Author-email: 
License: AGPL-3.0
Location: /opt/conda/lib/python3.10/site-packages
Editable project location: /usr/src/ultralytics
Requires: hub-sdk, matplotlib, numpy, opencv-python, pandas, pillow, psutil, py-cpuinfo, pyyaml, requests, scipy, seaborn, thop, torch, torchvision, tqdm
Required-by: 


In [7]:
! tensorboard --logdir ${ULTRALYTICS_DIR}/runs --port 6006 --host 0.0.0.0 # needs '--host 0.0.0.0'

In [1]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

results = model.train(data="/workspace/project/tmp/custom/custom.yaml", epochs=100, imgsz=416, device=[0])

New https://pypi.org/project/ultralytics/8.1.4 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.1 🚀 Python-3.10.13 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 2060 with Max-Q Design, 6144MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/workspace/project/tmp/custom/custom.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks

train: Scanning /workspace/project/tmp/custom/labels/train.cache... 42 images, 0 backgrounds, 5 corrupt: 100%|██████████| 42/42 [00:00<?, ?it/s]

train: WARNING ⚠️ /workspace/project/tmp/custom/images/train/ui_progressbar_1.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.024]
train: WARNING ⚠️ /workspace/project/tmp/custom/images/train/ui_progressbar_5.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1851]
train: WARNING ⚠️ /workspace/project/tmp/custom/images/train/ui_slider_0.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1587]
train: WARNING ⚠️ /workspace/project/tmp/custom/images/train/ui_slider_1.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0337]
train: WARNING ⚠️ /workspace/project/tmp/custom/images/train/ui_slider_9.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.012]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /workspace/project/tmp/custom/labels/val.cache... 6 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6/6 [00:00<?, ?it/s]


...

    100/100      1.06G      0.806      1.211     0.9592          5        416: 100%|██████████| 3/3 [00:01<00:00,  2.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  9.10it/s]

                   all          6          6      0.901          1      0.995      0.908



100 epochs completed in 0.057 hours.
Optimizer stripped from /usr/src/ultralytics/runs/detect/train7/weights/last.pt, 6.2MB
Optimizer stripped from /usr/src/ultralytics/runs/detect/train7/weights/best.pt, 6.2MB

Validating /usr/src/ultralytics/runs/detect/train7/weights/best.pt...
Ultralytics YOLOv8.1.1 🚀 Python-3.10.13 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 2060 with Max-Q Design, 6144MiB)
Model summary (fused): 168 layers, 3006818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  6.29it/s]


                   all          6          6      0.778          1      0.995       0.92
           lv_checkbox          6          2       0.83          1      0.995      0.796
              lv_label          6          1      0.734          1      0.995      0.995
             lv_slider          6          2      0.842          1      0.995      0.895
             lv_switch          6          1      0.706          1      0.995      0.995
Speed: 0.7ms preprocess, 7.0ms inference, 0.0ms loss, 6.0ms postprocess per image
Results saved to /usr/src/ultralytics/runs/detect/train7


In [4]:
# Perform validation using the validate function
model = YOLO('/usr/src/ultralytics/runs/detect/train7/weights/best.pt')
results = model.val()

Ultralytics YOLOv8.1.1 🚀 Python-3.10.13 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 2060 with Max-Q Design, 6144MiB)
Model summary (fused): 168 layers, 3006818 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /workspace/project/tmp/custom/labels/val.cache... 6 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6/6 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.29it/s]


                   all          6          6      0.781          1      0.995       0.92
           lv_checkbox          6          2      0.831          1      0.995      0.796
              lv_label          6          1      0.735          1      0.995      0.995
             lv_slider          6          2      0.852          1      0.995      0.895
             lv_switch          6          1      0.707          1      0.995      0.995
Speed: 0.9ms preprocess, 35.0ms inference, 0.0ms loss, 7.7ms postprocess per image
Results saved to /usr/src/ultralytics/runs/detect/val


In [5]:
results.box.map

0.9203749999999999

In [7]:
results.box.map50

0.995

In [8]:
results.box.map75

0.995

In [9]:
results.box.maps

array([    0.92037,       0.796,       0.995,      0.8955,       0.995,     0.92037])

In [10]:
from ultralytics.utils.benchmarks import benchmark

# Benchmark on GPU
benchmark(model='/usr/src/ultralytics/runs/detect/train7/weights/best.pt', data='/workspace/project/tmp/custom/custom.yaml', imgsz=416, half=False, device=[0])

Setup complete ✅ (4 CPUs, 7.8 GB RAM, 69.1/1006.9 GB disk)

Benchmarks complete for best.pt on /workspace/project/tmp/custom/custom.yaml at imgsz=416 (265.90s)
                   Format Status❔  Size (MB)  metrics/mAP50-95(B)  Inference time (ms/im)
0                 PyTorch       ✅        5.9               0.9204                   39.20
1             TorchScript       ✅       11.8               0.9017                   15.79
2                    ONNX       ✅       11.6               0.9017                   71.85
3                OpenVINO       ❌        0.0                  NaN                     NaN
4                TensorRT       ✅       14.5               0.9017                    8.12
5                  CoreML       ❌        0.0                  NaN                     NaN
6   TensorFlow SavedModel       ✅       29.1               0.9017                  107.75
7     TensorFlow GraphDef       ✅       11.6               0.9017                  145.40
8         TensorFlow Lite     

,Format,Status❔,Size (MB),metrics/mAP50-95(B),Inference time (ms/im)
0,PyTorch,✅,5.9,0.9204,39.20
1,TorchScript,✅,11.8,0.9017,15.79
2,ONNX,✅,11.6,0.9017,71.85
3,OpenVINO,❌,0.0,NaN,NaN
4,TensorRT,✅,14.5,0.9017,8.12
5,CoreML,❌,0.0,NaN,NaN
6,TensorFlow SavedModel,✅,29.1,0.9017,107.75
7,TensorFlow GraphDef,✅,11.6,0.9017,145.40
8,TensorFlow Lite,❌,0.0,NaN,NaN
9,TensorFlow Edge TPU,❌,0.0,NaN,NaN
